In [0]:
-- ============================================================
-- 03 - GOLD WAREHOUSE (SQL VERSION)
--  - dim_date
--  - dim_listing
--  - dim_host
--  - dim_neighbourhood
--  - fact_calendar
--  - fact_review
-- ============================================================

--------------------------------------------------
-- 0. Ensure schema
--------------------------------------------------
CREATE SCHEMA IF NOT EXISTS airbnb_project;
USE airbnb_project;

-- ============================================================
-- 1) DIM_DATE (STATIC) - DROP & RECREATE
-- ============================================================

DROP TABLE IF EXISTS airbnb_dim_date;

CREATE TABLE airbnb_dim_date
USING DELTA AS
WITH dates AS (
  SELECT sequence(
    to_date('2019-01-01'),
    to_date('2026-12-31'),
    interval 1 day
  ) AS d
)
SELECT
  explode(d)                                      AS date,
  CAST(date_format(date, 'yyyyMMdd') AS INT)      AS date_key,
  year(date)                                      AS year,
  month(date)                                     AS month,
  day(date)                                       AS day,
  weekofyear(date)                                AS week_of_year,
  date_format(date, 'E')                          AS day_name
FROM dates;

SELECT 'Created airbnb_dim_date' AS info;

-- ============================================================
-- 2) DIM_LISTING – TYPE 1 MERGE FROM STAGING
-- ============================================================

-- 2.1 Ensure target table exists with basic schema
CREATE TABLE IF NOT EXISTS airbnb_dim_listing (
  listing_id        BIGINT,
  host_id           BIGINT,
  name              STRING,
  host_name         STRING,
  neighbourhood     STRING,
  neighbourhood_group STRING,
  latitude          DOUBLE,
  longitude         DOUBLE,
  room_type         STRING,
  base_price        DOUBLE,
  snapshot_date     DATE
)
USING DELTA;

-- 2.2 Deduplicate stg_listings: latest snapshot per listing_id and MERGE
WITH latest_listing AS (
  SELECT
    listing_id,
    host_id,
    name,
    host_name,
    neighbourhood,
    neighbourhood_group,
    latitude,
    longitude,
    room_type,
    price,
    snapshot_date,
    ROW_NUMBER() OVER (
      PARTITION BY listing_id
      ORDER BY snapshot_date DESC
    ) AS rn
  FROM airbnb_stg_listings
),

dim_listing_current AS (
  SELECT
    CAST(listing_id AS BIGINT) AS listing_id,
    CAST(host_id   AS BIGINT)  AS host_id,
    name,
    host_name,
    neighbourhood,
    neighbourhood_group,
    latitude,
    longitude,
    room_type,
    price         AS base_price,
    snapshot_date
  FROM latest_listing
  WHERE rn = 1
)

MERGE INTO airbnb_dim_listing AS t
USING dim_listing_current AS s
ON t.listing_id = s.listing_id
WHEN MATCHED THEN
  UPDATE SET *
WHEN NOT MATCHED THEN
  INSERT *;

SELECT 'Upserted airbnb_dim_listing (Type 1)' AS info;

-- ============================================================
-- 3) DIM_HOST – REBUILD (CHEAP, SMALL)
-- ============================================================

CREATE OR REPLACE TABLE airbnb_dim_host
USING DELTA AS
SELECT
  host_id,
  FIRST(host_name, TRUE)                                  AS host_name,
  COUNT(DISTINCT listing_id)                              AS host_listings_count
FROM airbnb_stg_listings
GROUP BY host_id;

SELECT 'Rebuilt airbnb_dim_host' AS info;

-- ============================================================
-- 4) DIM_NEIGHBOURHOOD – REBUILD
-- ============================================================

CREATE OR REPLACE TABLE airbnb_dim_neighbourhood
USING DELTA AS
SELECT
  neighbourhood,
  neighbourhood_group
FROM airbnb_stg_neighbourhoods
GROUP BY neighbourhood, neighbourhood_group;

SELECT 'Rebuilt airbnb_dim_neighbourhood' AS info;

-- ============================================================
-- 5) FACT_CALENDAR – INCREMENTAL APPEND BY DATE
-- ============================================================

-- 5.1 Create table if not exists
CREATE TABLE IF NOT EXISTS airbnb_fact_calendar (
  listing_id        BIGINT,
  date_key          INT,
  date              DATE,
  is_available      INT,
  nightly_price     DOUBLE,
  snapshot_date     DATE
)
USING DELTA;

-- 5.2 Find max existing date in fact table
WITH max_existing AS (
  SELECT COALESCE(MAX(date), DATE '1900-01-01') AS max_date
  FROM airbnb_fact_calendar
),
fact_calendar_all AS (
  SELECT
    CAST(c.listing_id AS BIGINT)  AS listing_id,
    d.date_key,
    c.date,
    c.is_available,
    c.price                      AS nightly_price,
    c.snapshot_date
  FROM airbnb_stg_calendar c
  JOIN airbnb_dim_date d
    ON c.date = d.date
),
to_insert AS (
  SELECT f.*
  FROM fact_calendar_all f
  CROSS JOIN max_existing m
  WHERE f.date > m.max_date
)
INSERT INTO airbnb_fact_calendar
SELECT * FROM to_insert;

SELECT 'Fact_calendar incremental load done' AS info;

-- ============================================================
-- 6) FACT_REVIEW – INCREMENTAL APPEND BY REVIEW_DATE
-- ============================================================

-- 6.1 Create table if not exists
CREATE TABLE IF NOT EXISTS airbnb_fact_review (
  review_id      BIGINT,
  listing_id     BIGINT,
  date_key       INT,
  review_date    DATE,
  reviewer_name  STRING,
  comments       STRING,
  snapshot_date  DATE
)
USING DELTA;

-- 6.2 Find max existing review_date in fact
WITH max_existing AS (
  SELECT COALESCE(MAX(review_date), DATE '1900-01-01') AS max_rdate
  FROM airbnb_fact_review
),
fact_review_all AS (
  SELECT
    CAST(r.review_id   AS BIGINT) AS review_id,
    CAST(r.listing_id  AS BIGINT) AS listing_id,
    d.date_key,
    r.review_date,
    r.reviewer_name,
    r.comments,
    r.snapshot_date
  FROM airbnb_stg_reviews r
  JOIN airbnb_dim_date d
    ON r.review_date = d.date
),
to_insert AS (
  SELECT f.*
  FROM fact_review_all f
  CROSS JOIN max_existing m
  WHERE f.review_date > m.max_rdate
)
INSERT INTO airbnb_fact_review
SELECT * FROM to_insert;

SELECT 'Fact_review incremental load done' AS info;

-- ============================================================
-- DONE
-- ============================================================
SELECT '03 Gold warehouse SQL refresh complete' AS status;
